<a href="https://colab.research.google.com/github/SaqlainHussainShah/implementation-of-sdi/blob/master/K_fold_Separate_Notebook_B_3k_dropped_ben.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Steps for implementation
#### Reading CSV
#### Data Description
#### Data Cleaning
#### Data Normalization
#### Features
#### Applying KNN and NB
#### Confusion Matrix

In [0]:
#Importing Necessary Modules
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.metrics import accuracy_score

### Reading CSV

In [113]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [114]:
df1=pd.read_csv("/content/drive/My Drive/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
df2=pd.read_csv("/content/drive/My Drive/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
df3=pd.read_csv("/content/drive/My Drive/Friday-WorkingHours-Morning.pcap_ISCX.csv")
df4=pd.read_csv("/content/drive/My Drive/Monday-WorkingHours.pcap_ISCX.csv")
df5=pd.read_csv("/content/drive/My Drive/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
df6=pd.read_csv("/content/drive/My Drive/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
df7=pd.read_csv("/content/drive/My Drive/Tuesday-WorkingHours.pcap_ISCX.csv")
df8=pd.read_csv("/content/drive/My Drive/Wednesday-workingHours.pcap_ISCX.csv")


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8])

In [0]:
del df1,df2,df3,df4,df5,df6,df7,df8


In [0]:
# category B attacks
df_brute_force=df[df[' Label']=='Web Attack-Brute Force']
df_xss=df[df[' Label']=='Web Attack-XSS']
df_sql=df[df[' Label']=='Web Attack-Sql Injection']
#df_heartbleed=df[df[' Label']=='Heartbleed']
df_slowloris=df[df[' Label']=='DoS slowloris']
df_portscan=df[df[' Label']=='PortScan']
df_slow_http_test=df[df[' Label']=='DoS Slowhttptest']

In [0]:
# benign data
df_benign=df[df[' Label']=='BENIGN']

In [119]:
df_cat_B=pd.concat([df_brute_force,df_xss,df_sql,df_slowloris,df_portscan,df_slow_http_test, df_benign])
df_cat_B.groupby(' Label').first()


,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,...,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
Label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
BENIGN,192.168.10.5-104.16.207.165-54865-443-6,104.16.207.165,443,192.168.10.5,54865,6,7/7/2017 3:30,3,2,0,12.0,0.0,6.0,6.0,6.000000,0.000000,0.0,0.0,0.000000,0.000000,4000000,666666.6667,3.000000e+00,0.000000e+00,3.0,3.0,3.0,3.000000e+00,0.000000e+00,3.0,3.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,...,6.0,6.0,6.000000,0.000000,0.00000,0,0,0,0,1,0,0,0,0.0,9.000000,6.000000,0.000000,40,0,0,0,0,0,0,2,12,0,0,33,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DoS Slowhttptest,172.16.0.1-192.168.10.50-33372-80-6,172.16.0.1,33372,192.168.10.50,80,6,5/7/2017 10:15,16045,5,3,356.0,1964.0,356.0,0.0,71.200000,159.208040,1964.0,0.0,654.666667,1133.915929,144593.3313,498.597694,2.292143e+03,2.236413e+03,5415.0,73.0,16045.0,4.011250e+03,1.779768e+03,5415.0,1459.0,9788.0,4.894000e+03,9.461089e+02,5563.0,4225.0,0,0,0,0,...,0.0,1964.0,257.777778,650.575472,423248.44440,0,0,0,1,0,0,0,0,0.0,290.000000,71.200000,654.666667,168,0,0,0,0,0,0,5,356,3,1964,29200,235,1,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DoS slowloris,172.16.0.1-192.168.10.50-49631-80-6,172.16.0.1,49631,192.168.10.50,80,6,5/7/2017 2:24,5169956,8,6,1101.0,4222.0,410.0,0.0,137.625000,185.758628,3525.0,0.0,703.666667,1395.868284,1029.602573,2.707953414,3.976889e+05,1.368409e+06,4951173.0,112.0,218783.0,3.125471e+04,3.068629e+04,78311.0,219.0,5169833.0,1.033967e+06,2.218975e+06,5002421.0,840.0,0,0,0,0,...,0.0,3525.0,354.866667,895.115146,801231.12380,0,0,0,1,0,0,0,0,0.0,380.214286,137.625000,703.666667,172,0,0,0,0,0,0,8,1101,6,4222,8192,254,7,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PortScan,172.16.0.1-192.168.10.50-40620-80-6,172.16.0.1,40620,192.168.10.50,80,6,7/7/2017 1:05,5021059,6,5,703.0,1414.0,356.0,0.0,117.166667,181.536130,1050.0,0.0,282.800000,456.923626,421.6242032,2.190772903,5.021059e+05,1.568379e+06,4965658.0,19.0,55401.0,1.108020e+04,1.761267e+04,41863.0,19.0,5020928.0,1.255232e+06,2.499940e+06,5005133.0,1053.0,0,0,0,0,...,0.0,1050.0,176.416667,317.471103,100787.90150,0,0,0,1,0,0,0,0,0.0,192.454545,117.166667,282.800000,200,0,0,0,0,0,0,6,703,5,1414,29200,243,2,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Web Attack-Brute Force,172.16.0.1-192.168.10.50-44380-80-6,172.16.0.1,44380,192.168.10.50,80,6,6/7/2017 9:15,5185118,7,7,1022.0,2321.0,372.0,0.0,146.000000,184.078788,1047.0,0.0,331.571429,439.659284,644.7297824,2.700034985,3.988552e+05,1.372181e+06,4963956.0,4.0,221162.0,3.686033e+04,5.614102e+04,141434.0,4.0,5185004.0,8.641673e+05,2.027593e+06,5001548.0,879.0,0,0,0,0,...,0.0,1047.0,222.866667,331.323939,109775.55240,0,0,0,1,0,0,0,0,1.0,238.785714,146.000000,331.571429,232,0,0,0,0,0,0,7,1022,7,2321,29200,252,3,32,0.0,0.0,0

In [0]:
del df_brute_force,df_xss,df_sql,df_slowloris,df_portscan,df_slow_http_test, df_benign

In [0]:
df=df_cat_B.copy()

In [0]:
del df_cat_B

In [0]:
### remove some (Huge) benign data

df=df.iloc[0:-2000000]

In [124]:
df.groupby(' Label').first()

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,...,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
Label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
BENIGN,192.168.10.5-104.16.207.165-54865-443-6,104.16.207.165,443,192.168.10.5,54865,6,7/7/2017 3:30,3,2,0,12.0,0.0,6.0,6.0,6.000000,0.000000,0.0,0.0,0.000000,0.000000,4000000,666666.6667,3.000000e+00,0.000000e+00,3.0,3.0,3.0,3.000000e+00,0.000000e+00,3.0,3.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0,0,0,0,...,6.0,6.0,6.000000,0.000000,0.00000,0,0,0,0,1,0,0,0,0.0,9.000000,6.000000,0.000000,40,0,0,0,0,0,0,2,12,0,0,33,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DoS Slowhttptest,172.16.0.1-192.168.10.50-33372-80-6,172.16.0.1,33372,192.168.10.50,80,6,5/7/2017 10:15,16045,5,3,356.0,1964.0,356.0,0.0,71.200000,159.208040,1964.0,0.0,654.666667,1133.915929,144593.3313,498.597694,2.292143e+03,2.236413e+03,5415.0,73.0,16045.0,4.011250e+03,1.779768e+03,5415.0,1459.0,9788.0,4.894000e+03,9.461089e+02,5563.0,4225.0,0,0,0,0,...,0.0,1964.0,257.777778,650.575472,423248.44440,0,0,0,1,0,0,0,0,0.0,290.000000,71.200000,654.666667,168,0,0,0,0,0,0,5,356,3,1964,29200,235,1,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DoS slowloris,172.16.0.1-192.168.10.50-49631-80-6,172.16.0.1,49631,192.168.10.50,80,6,5/7/2017 2:24,5169956,8,6,1101.0,4222.0,410.0,0.0,137.625000,185.758628,3525.0,0.0,703.666667,1395.868284,1029.602573,2.707953414,3.976889e+05,1.368409e+06,4951173.0,112.0,218783.0,3.125471e+04,3.068629e+04,78311.0,219.0,5169833.0,1.033967e+06,2.218975e+06,5002421.0,840.0,0,0,0,0,...,0.0,3525.0,354.866667,895.115146,801231.12380,0,0,0,1,0,0,0,0,0.0,380.214286,137.625000,703.666667,172,0,0,0,0,0,0,8,1101,6,4222,8192,254,7,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PortScan,172.16.0.1-192.168.10.50-40620-80-6,172.16.0.1,40620,192.168.10.50,80,6,7/7/2017 1:05,5021059,6,5,703.0,1414.0,356.0,0.0,117.166667,181.536130,1050.0,0.0,282.800000,456.923626,421.6242032,2.190772903,5.021059e+05,1.568379e+06,4965658.0,19.0,55401.0,1.108020e+04,1.761267e+04,41863.0,19.0,5020928.0,1.255232e+06,2.499940e+06,5005133.0,1053.0,0,0,0,0,...,0.0,1050.0,176.416667,317.471103,100787.90150,0,0,0,1,0,0,0,0,0.0,192.454545,117.166667,282.800000,200,0,0,0,0,0,0,6,703,5,1414,29200,243,2,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Web Attack-Brute Force,172.16.0.1-192.168.10.50-44380-80-6,172.16.0.1,44380,192.168.10.50,80,6,6/7/2017 9:15,5185118,7,7,1022.0,2321.0,372.0,0.0,146.000000,184.078788,1047.0,0.0,331.571429,439.659284,644.7297824,2.700034985,3.988552e+05,1.372181e+06,4963956.0,4.0,221162.0,3.686033e+04,5.614102e+04,141434.0,4.0,5185004.0,8.641673e+05,2.027593e+06,5001548.0,879.0,0,0,0,0,...,0.0,1047.0,222.866667,331.323939,109775.55240,0,0,0,1,0,0,0,0,1.0,238.785714,146.000000,331.571429,232,0,0,0,0,0,0,7,1022,7,2321,29200,252,3,32,0.0,0.0,0

### Data Description

In [125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 445502 entries, 12637 to 47843
Data columns (total 85 columns):
Flow ID                         445502 non-null object
 Source IP                      445502 non-null object
 Source Port                    445502 non-null int64
 Destination IP                 445502 non-null object
 Destination Port               445502 non-null int64
 Protocol                       445502 non-null int64
 Timestamp                      445502 non-null object
 Flow Duration                  445502 non-null int64
 Total Fwd Packets              445502 non-null int64
 Total Backward Packets         445502 non-null int64
Total Length of Fwd Packets     445502 non-null float64
 Total Length of Bwd Packets    445502 non-null float64
 Fwd Packet Length Max          445502 non-null float64
 Fwd Packet Length Min          445502 non-null float64
 Fwd Packet Length Mean         445502 non-null float64
 Fwd Packet Length Std          445502 non-null float64
Bwd Pa

In [126]:
df.describe()

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,445502.000000,445502.000000,445502.000000,4.455020e+05,445502.000000,445502.000000,4.455020e+05,4.455020e+05,445502.000000,445502.000000,445502.000000,445502.000000,445502.000000,445502.000000,445502.000000,445502.000000,4.455020e+05,4.455020e+05,4.455020e+05,4.455020e+05,4.455020e+05,4.455020e+05,4.455020e+05,4.455020e+05,4.455020e+05,4.455020e+05,4.455020e+05,4.455020e+05,4.455020e+05,4.455020e+05,445502.000000,445502.0,445502.0,445502.0,4.455020e+05,4.455020e+05,4.455020e+05,4.455020e+05,445502.000000,445502.000000,445502.000000,445502.000000,4.455020e+05,445502.000000,445502.000000,445502.000000,445502.000000,445502.000000,445502.000000,445502.0,445502.000000,445502.000000,445502.00000,445502.000000,445502.000000,4.455020e+05,445502.0,445502.0,445502.0,445502.0,445502.0,445502.0,445502.000000,4.455020e+05,445502.000000,4.455020e+05,445502.000000,445502.000000,445502.000000,445502.000000,4.455020e+05,4.455020e+05,4.455020e+05,4.455020e+05,4.455020e+05,4.455020e+05,4.455020e+05,4.455020e+05
mean,39991.712329,10364.924003,8.876144,9.778307e+06,7.335383,8.424021,7.219514e+02,1.222336e+04,344.718084,24.341130,105.386602,129.846005,232.560806,32.216248,95.103356,70.774018,8.817163e+05,1.787845e+06,5.137229e+06,5.486430e+04,9.359108e+06,1.840969e+06,1.645263e+06,4.921932e+06,9.342667e+05,8.676195e+06,1.870075e+06,1.424069e+06,4.550487e+06,1.058414e+06,0.041903,0.0,0.0,0.0,1.772000e+02,1.910565e+02,3.875578e+04,1.521094e+04,12.781727,502.396770,94.404356,154.247283,2.239331e+05,0.008236,0.041903,0.000148,0.519998,0.213855,0.112415,0.0,0.000148,0.959116,106.95590,105.386602,95.103356,1.772000e+02,0.0,0.0,0.0,0.0,0.0,0.0,7.335383,7.219514e+02,8.424021,1.222529e+04,9575.602017,1401.380306,5.099149,27.591477,1.322413e+05,5.090450e+04,1.998712e+05,1.001619e+05,4.096059e+06,7.672892e+05,4.749581e+06,3.512856e+06
std,21680.276240,18924.345982,4.838538,2.786342e+07,717.242313,967.021861,5.263111e+03,2.161567e+06,1382.044036,127.822352,374.479127,584.272098,656.998069,61.216611,230.377123,220.188522,3.520505e+06,6.441703e+06,1.686241e+07,1.165530e+06,2.780574e+07,9.115615e+06,6.608161e+06,1.733720e+07,8.541692e+06,2.680617e+07,9.684931e+06,6.116927e+06,1.665941e+07,8.773448e+06,0.200368,0.0,0.0,0.0,1.508941e+04,1.938922e+04,1.674198e+05,5.468596e+04,23.989047,1485.701744,216.004934,447.380471,1.157946e+06,0.090376,0.200368,0.012171,0.499600,0.410026,0.315876,0.0,0.012171,1.020183,236.55354,374.479127,230.377123,1.508941e+04,0.0,0.0,0.0,0.0,0.0,0.0,717.242313,5.263111e+03,967.021861,2.161985e+06,14104.580217,7203.732902,684.678360,7.799013,8.483649e+05,4.637452e+05,1.139548e+06,7.803129e+05,1.481516e+07,5.472717e+06,1.666315e+07,1.408908e+07
min,0.000000,0.00000

In [127]:
df.columns

Index(['Flow ID', ' Source IP', ' Source Port', ' Destination IP',
       ' Destination Port', ' Protocol', ' Timestamp', ' Flow Duration',
       ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
       ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
       ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean',
       ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags',
       ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags',
       ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s',
       ' Bwd Packets/s', ' Min Packet Length', ' Max Pa

### Data Cleaning

In [128]:
df2=df[df.columns[7:-1]]
df2[' Label']=df[' Label']
df=df2
df2=[]
del df2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [129]:
len(df.columns)

78

In [0]:
df=df.dropna( axis=0, how='any')
df=df.replace(',,', np.nan, inplace=False)
df=df.drop(columns=[' Fwd Header Length.1'], axis=1, inplace=False)

In [0]:
df.replace("Infinity", 0, inplace=True)
df['Flow Bytes/s'].replace("Infinity", 0,inplace=True)
df[" Flow Packets/s"].replace("Infinity", 0, inplace=True)
df[" Flow Packets/s"].replace(np.nan, 0, inplace=True)
df['Flow Bytes/s'].replace(np.nan, 0,inplace=True)


df["Bwd Avg Bulk Rate"].replace("Infinity", 0, inplace=True)
df["Bwd Avg Bulk Rate"].replace(",,", 0, inplace=True)
df["Bwd Avg Bulk Rate"].replace(np.nan, 0, inplace=True)

df[" Bwd Avg Packets/Bulk"].replace("Infinity", 0, inplace=True)
df[" Bwd Avg Packets/Bulk"].replace(",,", 0, inplace=True)
df[" Bwd Avg Packets/Bulk"].replace(np.nan, 0, inplace=True)


df[" Bwd Avg Bytes/Bulk"].replace("Infinity", 0, inplace=True)
df[" Bwd Avg Bytes/Bulk"].replace(",,", 0, inplace=True)
df[" Bwd Avg Bytes/Bulk"].replace(np.nan, 0, inplace=True)


df[" Fwd Avg Bulk Rate"].replace("Infinity", 0, inplace=True)
df[" Fwd Avg Bulk Rate"].replace(",,", 0, inplace=True)
df[" Fwd Avg Bulk Rate"].replace(np.nan, 0, inplace=True)


df[" Fwd Avg Packets/Bulk"].replace("Infinity", 0, inplace=True)
df[" Fwd Avg Packets/Bulk"].replace(",,", 0, inplace=True)
df[" Fwd Avg Packets/Bulk"].replace(np.nan, 0, inplace=True)


df["Fwd Avg Bytes/Bulk"].replace("Infinity", 0, inplace=True)
df["Fwd Avg Bytes/Bulk"].replace(",,", 0, inplace=True)
df["Fwd Avg Bytes/Bulk"].replace(np.nan, 0, inplace=True)


df[" CWE Flag Count"].replace("Infinity", 0, inplace=True)
df[" CWE Flag Count"].replace(",,", 0, inplace=True)
df[" CWE Flag Count"].replace(np.nan, 0, inplace=True)

df[" Bwd URG Flags"].replace("Infinity", 0, inplace=True)
df[" Bwd URG Flags"].replace(",,", 0, inplace=True)
df[" Bwd URG Flags"].replace(np.nan, 0, inplace=True)

df[" Bwd PSH Flags"].replace("Infinity", 0, inplace=True)
df[" Bwd PSH Flags"].replace(",,", 0, inplace=True)
df[" Bwd PSH Flags"].replace(np.nan, 0, inplace=True)

df[" Fwd URG Flags"].replace("Infinity", 0, inplace=True)
df[" Fwd URG Flags"].replace(",,", 0, inplace=True)
df[" Fwd URG Flags"].replace(np.nan, 0, inplace=True)

In [0]:
df["Flow Bytes/s"]=df["Flow Bytes/s"].astype("float64")
df[' Flow Packets/s']=df[" Flow Packets/s"].astype("float64")

In [0]:
df['Bwd Avg Bulk Rate']=df["Bwd Avg Bulk Rate"].astype("float64")
df[' Bwd Avg Packets/Bulk']=df[" Bwd Avg Packets/Bulk"].astype("float64")
df[' Bwd Avg Bytes/Bulk']=df[" Bwd Avg Bytes/Bulk"].astype("float64")
df[' Fwd Avg Bulk Rate']=df[" Fwd Avg Bulk Rate"].astype("float64")
df[' Fwd Avg Packets/Bulk']=df[" Fwd Avg Packets/Bulk"].astype("float64")
df['Fwd Avg Bytes/Bulk']=df["Fwd Avg Bytes/Bulk"].astype("float64")
df[' CWE Flag Count']=df[" CWE Flag Count"].astype("float64")
df[' Bwd URG Flags']=df[" Bwd URG Flags"].astype("float64")
df[' Bwd PSH Flags']=df[" Bwd PSH Flags"].astype("float64")
df[' Fwd URG Flags']=df[" Fwd URG Flags"].astype("float64")

In [0]:
pd.set_option('display.max_rows', df.shape[0])

In [0]:
df.replace('Infinity',0.0, inplace=True)

In [0]:
df.replace('NaN',0.0, inplace=True)

In [137]:
df.columns

Index([' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
       ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
       ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean',
       ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags',
       ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags',
       ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s',
       ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length',
       ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance',
       'FIN Flag Count', ' SY

In [0]:



X=df[df.columns[0:-1]]
y=df[df.columns[-1]]

del df

### Data Normalization

In [0]:
from scipy import stats

In [140]:


cols = list(X.columns)
for col in cols:
    X[col] = stats.zscore(X[col])



/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:2315: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


In [141]:
X.head()

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
12637,-0.164864,-0.000468,-0.001473,0.057000,-0.004581,0.019725,-0.190436,0.108433,0.092805,1.239584,-0.526287,1.026389,1.675252,-0.048638,-0.281962,-0.137168,-0.064542,-0.010294,-0.047070,-0.328647,-0.197921,-0.240487,-0.275746,-0.109380,-0.130255,-0.103872,0.098648,0.027058,-0.120542,-0.209138,NaN,NaN,NaN,0.003631,0.002111,-0.231488,-0.278135,-0.532836,0.366533,0.594679,0.395777,-0.098595,-0.091129,-0.209138,-0.012173,0.960713,-0.521492,-0.355879,NaN,-0.012173,0.040044,0.557254,0.108433,1.026389,NaN,NaN,NaN,NaN,NaN,NaN,-0.000468,0.057000,-0.001473,-0.004581,1.391426,-0.159527,-0.003066,0.565302,-0.155883,-0.109772,-0.175401,-0.128365,-0.276487,-0.140207,-0.285045,-0.24934
12643,-0.169448,0.003714,-0.001473,0.002661,-0.001084,0.036366,-0.190436,-0.084897,0.044207,6.257736,-0.526287,5.713622,7.126149,-0.048588,-0.281959,-0.160678,-0.085177,-0.010343,-0.047043,-0.154723,-0.140321,0.000782,0.002359,-0.109358,-0.321579,-0.192131,-0.231088,-0.271859,-0.120638,-0.209138,NaN,NaN,NaN,0.009993,0.002111,-0.231485,-0.278135,-0.532836,2.585629,2.293263,2.316084,1.030446,-0.091129,-0.209138,-0.012173,0.960713,-0.521492,-0.355879,NaN,-0.012173,-0.940158,2.187650,-0.084897,5.713622,NaN,NaN,NaN,NaN,NaN,NaN,0.003714,0.002661,-0.001473,-0.001085,1.391426,-0.160777,-0.004527,0.565302,-0.155883,-0.109772,-0.175401,-0.128365,-0.276487,-0.140207,-0.285045,-0.24934
12700,-0.350946,-0.008833,-0.007677,-0.137176,-0.005655,-0.249435,-0.190436,-0.281431,-0.222243,-0.353987,-0.526287,-0.412831,-0.321436,-0.048660,-0.152566,-0.250437,-0.277552,-0.304661,-0.047004,-0.336601,-0.201964,-0.248983,-0.283904,-0.109381,-0.323675,-0.193097,-0.232816,-0.273157,-0.120642,-0.209138,NaN,NaN,NaN,-0.009623,-0.008204,-0.157758,-0.052410,-0.532836,-0.338166,-0.437063,-0.344791,-0.193395,-0.091129,-0.209138,-0.012173,-1.040894,1.917576,2.809947,NaN,-0.012173,0.040044,-0.452159,-0.281431,-0.412831,NaN,NaN,NaN,NaN,NaN,NaN,-0.008833,-0.137176,-0.007677,-0.005655,-0.659784,-0.159527,-0.007448,0.565302,-0.155883,-0.109772,-0.175401,-0.128365,-0.276487,-0.140207,-0.285045,-0.24934
12712,-0.161777,-0.000468,-0.003541,0.046930,-0.004993,0.156475,-0.190436,0.088215,0.189373,1.268503,-0.526287,0.828582,1.784189,-0.048644,-0.281964,-0.114349,-0.046433,-0.010415,-0.047034,-0.147036,-0.105592,0.054135,0.002289,-0.109343,-0.313596,-0.186123,-0.211301,-0.257253,-0.120524,-0.209138,NaN,NaN,NaN,0.003631,-0.001190,-0.231488,-0.278143,-0.532836,0.379321,0.417242,0.392967,-0.099313,-0.091129,-0.209138,-0.012173,0.960713,-0.521492,-0.355879,NaN,-0.012173,-0.940158,0.392943,0.088215,0.828582,NaN,NaN,NaN,NaN,NaN,NaN,-0.000468,0.046930,-0.003541,-0.004993,1.391426,-0.160638,-0.004527,0.565302,-0.155883,-0.109772,-0.175401,-0.128365,-0.276487,-0.140207

### Feature Selection

In [0]:
features=[" Fwd Packet Length Max"," Flow IAT Std"," Fwd Packet Length Std" ,"Fwd IAT Total",' Flow Packets/s',' Flow Duration', " Fwd Packet Length Mean", " Total Length of Bwd Packets", "Flow Bytes/s", "Total Length of Fwd Packets", " Flow IAT Mean", " Bwd Packet Length Mean",  " Flow IAT Max", " Bwd Packet Length Std", ' Total Fwd Packets', ' Total Backward Packets']

In [143]:
X=X[features].copy()
X.head()

,Fwd Packet Length Max,Flow IAT Std,Fwd Packet Length Std,Fwd IAT Total,Flow Packets/s,Flow Duration,Fwd Packet Length Mean,Total Length of Bwd Packets,Flow Bytes/s,Total Length of Fwd Packets,Flow IAT Mean,Bwd Packet Length Mean,Flow IAT Max,Bwd Packet Length Std,Total Fwd Packets,Total Backward Packets
12637,0.019725,-0.064542,0.092805,-0.328647,-0.281962,-0.164864,0.108433,-0.004581,-0.048638,0.057000,-0.137168,1.026389,-0.010294,1.675252,-0.000468,-0.001473
12643,0.036366,-0.085177,0.044207,-0.154723,-0.281959,-0.169448,-0.084897,-0.001084,-0.048588,0.002661,-0.160678,5.713622,-0.010343,7.126149,0.003714,-0.001473
12700,-0.249435,-0.277552,-0.222243,-0.336601,-0.152566,-0.350946,-0.281431,-0.005655,-0.048660,-0.137176,-0.250437,-0.412831,-0.304661,-0.321436,-0.008833,-0.007677
12712,0.156475,-0.046433,0.189373,-0.147036,-0.281964,-0.161777,0.088215,-0.004993,-0.048644,0.046930,-0.114349,0.828582,-0.010415,1.784189,-0.000468,-0.003541
12791,0.159369,-0.034247,0.191381,-0.156044,-0.281965,-0.170767,0.089741,-0.004993,-0.048643,0.047690,-0.107853,1.138935,-0.010371,1.960803,-0.000468,-0.004575


In [145]:
len(X.columns)

16

### Applying Algorithms

In [0]:
#Run if not using K-Fold cross validation
from sklearn.model_selection import train_test_split     # import module for train test split

In [0]:
#Run if not using K-Fold cross validation
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2, random_state=10)

#### KNN

In [0]:
#Run if not using K-Fold cross validation
from sklearn.neighbors import KNeighborsClassifier

In [0]:
#Run if not using K-Fold cross validation
t1=time.time()

for i in range(1,len(X_train.columns)+1):
    knn=KNeighborsClassifier(n_neighbors=i)
    model_knn=knn.fit(X_train,y_train)
    yhat=model_knn.predict(X_test)
    print("for " , i,  " as K, accuracy is : ", accuracy_score(y_test, yhat))
t2=time.time()
print(" time for ", i ," k's :: ", (t2-t1)/60 , " minutes")

for  1  as K, accuracy is :  0.998781233449007
for  2  as K, accuracy is :  0.9987035268568297
for  3  as K, accuracy is :  0.9988364460276594
for  4  as K, accuracy is :  0.998807817283173
for  5  as K, accuracy is :  0.9988323562070185
for  6  as K, accuracy is :  0.9988200867450958
for  7  as K, accuracy is :  0.9988405358483003
for  8  as K, accuracy is :  0.9988303112966981
for  9  as K, accuracy is :  0.9988364460276594
for  10  as K, accuracy is :  0.9988180418347753
for  11  as K, accuracy is :  0.9987996376418912
for  12  as K, accuracy is :  0.998785323269648
for  13  as K, accuracy is :  0.9987607843458025
for  14  as K, accuracy is :  0.9987607843458025
for  15  as K, accuracy is :  0.9987260208703548
for  16  as K, accuracy is :  0.9987260208703548
 time for  16  k's ::  61.21009658972422  minutes


#### K Fold Cross Validation and KNN

In [0]:
df=X.copy()
df[' Label']=y.values

In [0]:
df.reset_index(inplace=True)

In [148]:
df2=df[df.columns[1:-1]]
df2[' Label']=df[df.columns[-1]]
df=df2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
del df2

In [150]:
df.head()


,Fwd Packet Length Max,Flow IAT Std,Fwd Packet Length Std,Fwd IAT Total,Flow Packets/s,Flow Duration,Fwd Packet Length Mean,Total Length of Bwd Packets,Flow Bytes/s,Total Length of Fwd Packets,Flow IAT Mean,Bwd Packet Length Mean,Flow IAT Max,Bwd Packet Length Std,Total Fwd Packets,Total Backward Packets,Label
0,0.019725,-0.064542,0.092805,-0.328647,-0.281962,-0.164864,0.108433,-0.004581,-0.048638,0.057000,-0.137168,1.026389,-0.010294,1.675252,-0.000468,-0.001473,Web Attack-Brute Force
1,0.036366,-0.085177,0.044207,-0.154723,-0.281959,-0.169448,-0.084897,-0.001084,-0.048588,0.002661,-0.160678,5.713622,-0.010343,7.126149,0.003714,-0.001473,Web Attack-Brute Force
2,-0.249435,-0.277552,-0.222243,-0.336601,-0.152566,-0.350946,-0.281431,-0.005655,-0.048660,-0.137176,-0.250437,-0.412831,-0.304661,-0.321436,-0.008833,-0.007677,Web Attack-Brute Force
3,0.156475,-0.046433,0.189373,-0.147036,-0.281964,-0.161777,0.088215,-0.004993,-0.048644,0.046930,-0.114349,0.828582,-0.010415,1.784189,-0.000468,-0.003541,Web Attack-Brute Force
4,0.159369,-0.034247,0.191381,-0.156044,-0.281965,-0.170767,0.089741,-0.004993,-0.048643,0.047690,-0.107853,1.138935,-0.010371,1.960803,-0.000468,-0.004575,Web Attack-Brute Force


In [0]:
X=df[df.columns[0:-1]]
y=df[df.columns[-1]]

del df


In [152]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445475 entries, 0 to 445474
Data columns (total 16 columns):
 Fwd Packet Length Max          445475 non-null float64
 Flow IAT Std                   445475 non-null float64
 Fwd Packet Length Std          445475 non-null float64
Fwd IAT Total                   445475 non-null float64
 Flow Packets/s                 445475 non-null float64
 Flow Duration                  445475 non-null float64
 Fwd Packet Length Mean         445475 non-null float64
 Total Length of Bwd Packets    445475 non-null float64
Flow Bytes/s                    445475 non-null float64
Total Length of Fwd Packets     445475 non-null float64
 Flow IAT Mean                  445475 non-null float64
 Bwd Packet Length Mean         445475 non-null float64
 Flow IAT Max                   445475 non-null float64
 Bwd Packet Length Std          445475 non-null float64
 Total Fwd Packets              445475 non-null float64
 Total Backward Packets         445475 non-null f

In [153]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
i=0
scores=np.zeros(10)
t1=time.time()
kf = KFold(n_splits=10)

for train_index, test_index in kf.split(X):
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]
  
  
   #  can use loop here
     
  knn=KNeighborsClassifier(n_neighbors=3)
  model_knn=knn.fit(X_train,y_train)
  yhat=model_knn.predict(X_test)
  
  scores[i]=accuracy_score(y_test, yhat)
  print("for " , i,  " as K, accuracy is : ", scores[i])
  i=i+1
  
  
#     X_train=train[train.columns[0:-1]]
#     y_train=train[train.columns[-1]]
    
#     X_test=test[test.columns[0:-1]]
#     y_test=test[test.columns[-1]]


t2=time.time()
print(" time for ", i ," cross validations :: ", (t2-t1)/60 , " minutes")
    
   
     
     
    
     
     
     
    

for  0  as K, accuracy is :  0.8206204543413845
for  1  as K, accuracy is :  0.9987204812786208
for  2  as K, accuracy is :  0.9996857322438718
for  3  as K, accuracy is :  0.8748765376672354
for  4  as K, accuracy is :  0.9992816737002783
for  5  as K, accuracy is :  0.9984735223471839
for  6  as K, accuracy is :  0.9973511123083485
for  7  as K, accuracy is :  0.9991469683704851
for  8  as K, accuracy is :  0.9931308505623274
for  9  as K, accuracy is :  0.9971041820998047
 time for  10  cross validations ::  16.21888049840927  minutes


In [154]:
print(" time in hours :::: ", (t2-t1)/(60*60), " hours")

 time in hours ::::  0.2703146749734879  hours


In [0]:
average_accuracy=scores.mean()
print("average accuracy of K fold KNN :: ", average_accuracy)

average accuracy of K fold KNN ::  0.9164250419191875


#### Save Model

In [0]:
import pickle
filename = 'drive/My Drive/model_cross_validation_cat_B_3k.sav'

In [0]:
pickle.dump(model_knn, open(filename, 'wb'))

In [0]:
# to load model
model_knn = pickle.load(open(filename, 'rb'))
y_hat=model_knn.predict(X_test)

#### Naive Bayes

##### Gaussian Naive Bayes

In [0]:
from sklearn.naive_bayes import GaussianNB

In [60]:
t1=time.time()

gnb = GaussianNB()
model_gnb = gnb.fit(X_train, y_train)
yhat_gnb=model_gnb.predict(X_test)

print("Accuracy :: ", accuracy_score(y_test,yhat))

t2=time.time()
print(" time for Naive Baye's :: ", (t2-t1) , " seconds")

Accuracy ::  0.998245463357177
 time for Naive Baye's ::  4.744645595550537  seconds


##### Bernoulli Naive Bayes

In [0]:
from sklearn.naive_bayes import BernoulliNB

In [62]:


clf = BernoulliNB()
clf.fit(X_train, y_train)
yhat_bnb=clf.predict(X_test)
print("Accuracy ::: ", accuracy_score(y_test,yhat_bnb))



Accuracy :::  0.33908772274231214


### Confusion Matrix

In [155]:
y_test_arr=y_test.as_matrix()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [0]:
true={}
false={}
not_detected={}

for x in range(len(y_test_arr)):
    if y_test_arr[x]==yhat[x]:
      if y_test_arr[x] in true:
        true[y_test_arr[x]]=true[y_test_arr[x]]+1
      else:
        true[y_test_arr[x]]=1
    elif y_test_arr[x]!=yhat[x]:
      if yhat[x] in false:
        false[yhat[x]]=false[yhat[x]]+1

        if y_test_arr[x] in not_detected:
          not_detected[y_test_arr[x]]=not_detected[y_test_arr[x]]+1
        else:
          not_detected[y_test_arr[x]]=1

      else:
        false[yhat[x]]=1

        if y_test_arr[x] in not_detected:
          not_detected[y_test_arr[x]]=not_detected[y_test_arr[x]]+1
        else:
          not_detected[y_test_arr[x]]=1

In [160]:
true

{'BENIGN': 44418}

In [161]:
false

{'DoS Slowhttptest': 17,
 'DoS slowloris': 37,
 'PortScan': 42,
 'Web Attack-Brute Force': 31,
 'Web Attack-Sql Injection': 2}

In [162]:
not_detected

{'BENIGN': 129}

In [0]:
# CHeck for naive bayes


true_gnb={}
false_gnb={}
not_detected_gnb={}

for x in range(len(y_test_arr)):
    if y_test_arr[x]==yhat_gnb[x]:
      if y_test_arr[x] in true_gnb:
        true_gnb[y_test_arr[x]]=true_gnb[y_test_arr[x]]+1
      else:
        true_gnb[y_test_arr[x]]=1
    elif y_test_arr[x]!=yhat_gnb[x]:
      if yhat_gnb[x] in false_gnb:
        false_gnb[yhat_gnb[x]]=false_gnb[yhat_gnb[x]]+1

        if y_test_arr[x] in not_detected_gnb:
          not_detected_gnb[y_test_arr[x]]=not_detected_gnb[y_test_arr[x]]+1
        else:
          not_detected_gnb[y_test_arr[x]]=1

      else:
        false_gnb[yhat_gnb[x]]=1

        if y_test_arr[x] in not_detected_gnb:
          not_detected_gnb[y_test_arr[x]]=not_detected_gnb[y_test_arr[x]]+1
        else:
          not_detected_gnb[y_test_arr[x]]=1

In [69]:
true_gnb

{'BENIGN': 25249}

In [70]:
false_gnb

{'DoS Slowhttptest': 35972,
 'DoS slowloris': 85027,
 'PortScan': 52982,
 'Web Attack-Brute Force': 4442,
 'Web Attack-Sql Injection': 40494,
 'Web Attack-XSS': 343}

In [71]:
not_detected_gnb

{'BENIGN': 219260}